## imports:

In [63]:
from geometry import forward_backward_consistency_check
import os
import numpy as np
import torch
from torchvision.transforms import v2

## Paths:

In [91]:
sources = "../../sources"
images_dir = sources+"/images"
flow_res = sources+"/flows"
m_dir = flow_res + "/M"
flow_res_fwd = flow_res + "/fwd"
flow_res_bwd = flow_res + "/bwd"
mod = images_dir + "/demo_mod"
ref = images_dir + "/demo_ref"
Xm1 = mod + "/openimage_0995.png"
Xo1 = ref + "/openimage_0995.png"
res_example = images_dir + "/res_example"
flownet = flow_res + "/FlowNet2.flo"
spynet = flow_res + "/openimage_0995.flo"
openimage_0995_res = res_example + "/openimage_0995_ref_like.png"
checkpoints = "../../checkpoints"
flownet2_checkpoint = checkpoints + "/FlowNet2/FlowNet2_checkpoint.pth.tar"

## create optical flow via SpyNet:

In [92]:
!python SpyNet/pred.py --model sintel-final --one $mod --two $ref --out $flow_res_fwd

/home/din/miniconda3/envs/buna/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608935911/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


In [93]:
!python SpyNet/pred.py --model sintel-final --one $ref --two $mod --out $flow_res_bwd

/home/din/miniconda3/envs/buna/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608935911/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


## create optical flow via FlowNet2:

In [94]:
#add checkpoints download in the readme and run FlowNet2/main.py

In [95]:
def make_files_list(args_strOne,args_strTwo):
    dir_mod = args_strOne    
    dir_ref = args_strTwo
    path_ref=[]
    path_mod=[]
    for im in os.listdir(dir_ref):
        path_ref+=[os.path.join(dir_ref,im)]
    for im in os.listdir(dir_mod):
        path_mod+=[os.path.join(dir_mod,im)]    
    path_ref.sort()
    path_mod.sort()
    path_lst = list(zip(path_mod,path_ref))
    return path_lst

## Flow Consistency Mask

In [106]:
#make utils dir and storage all the helper function their.
def readFlow(fn):
    """ Read .flo file in Middlebury format"""
    # Code adapted from:
    # http://stackoverflow.com/questions/28013200/reading-middlebury-flow-files-with-python-bytes-array-numpy

    # WARNING: this will work on little-endian architectures (eg Intel x86) only!
    # print 'fn = %s'%(fn)
    with open(fn, 'rb') as f:
        magic = np.fromfile(f, np.float32, count=1)
        if 202021.25 != magic:
            print('Magic number incorrect. Invalid .flo file')
            return None
        else:
            w = np.fromfile(f, np.int32, count=1)
            h = np.fromfile(f, np.int32, count=1)
            # print 'Reading %d x %d flo file\n' % (w, h)
            data = np.fromfile(f, np.float32, count=2 * int(w) * int(h))
            # Reshape testdata into 3D array (columns, rows, bands)
            # The reshape here is for visualization, the original code is (w,h,2)
            return np.resize(data, (int(h), int(w), 2))
def toTensor(x_numpy,type =torch.float):
    res = torch.tensor(x_numpy,dtype = type)
    return res.permute(2,0,1).unsqueeze(0).cuda()
G = v2.GaussianBlur(kernel_size=(7, 7), sigma=(0.1, 5.))
import numpy as np
import torch

class SeedSetter:
    def __init__(self, seed):
        self.seed = seed

    def __enter__(self):
        self.numpy_state = np.random.get_state()
        self.torch_state = torch.get_rng_state()

        np.random.seed(self.seed)
        torch.manual_seed(self.seed)

    def __exit__(self, exc_type, exc_val, exc_tb):
        np.random.set_state(self.numpy_state)
        torch.set_rng_state(self.torch_state)

# Usage with a 'with' statement
seed_value = 42

In [107]:
path_lst = make_files_list(flow_res_fwd,flow_res_bwd)
for pair in path_lst:
    fwd_name , bwd_name = pair
    fwd = toTensor(readFlow(fwd_name))
    bwd = toTensor(readFlow(bwd_name))
    M_inconsistent , _ = forward_backward_consistency_check(fwd,bwd,alpha=0.85,beta=0.085)
    with SeedSetter(seed_value):
        M = 1 - G(M_inconsistent)
    #TO DO: save M in source dir
    name_img = fwd_name[:-4]
    name_img = name_img.split('/')[-1]
    flow_path = m_dir + "/" + name_img + ".pt"
    torch.save(M, flow_path)
#    tmp_M=torch.load(flow_path)
#    diff = tmp_M-M
#    print(torch.sum(diff))

    

tensor(0., device='cuda:0')
tensor(0., device='cuda:0')


/tmp/ipykernel_2565/2790470663.py:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  data = np.fromfile(f, np.float32, count=2 * int(w) * int(h))
/tmp/ipykernel_2565/2790470663.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return np.resize(data, (int(h), int(w), 2))


In [101]:
#

'../../sources/flows/M/openimage_0995.pt'